In [1]:
import torch
from torchvision import datasets, transforms

In [2]:
import syft as sy
hook = sy.TorchHook(torch)

# Create a couple of workers
bob = sy.VirtualWorker(hook, id="bob")  
alice = sy.VirtualWorker(hook, id="alice")

/home/akshay/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 1000
        self.epochs = 10
        self.lr = 0.01
        self.momentum = 0.5
        self.no_cuda = False
        self.seed = 1
        self.log_interval = 30
        self.save_model = False

args = Arguments()

torch.manual_seed(args.seed)

In [4]:
use_cuda = not args.no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

cuda


In [6]:
# FederatedDataLoader 
# we distribute the dataset across all the workers, it's now a FederatedDataset
transform=transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.1307,), (0.3081,))]) 
mnist_trainset = datasets.MNIST('~/.pytorch/F_MNIST_data/', train=True, download=True, transform=transform).federate((bob, alice))
mnist_testset = datasets.MNIST('~/.pytorch/F_MNIST_data/', train=False, download=True, transform=transform)

federated_train_loader = sy.FederatedDataLoader(mnist_trainset, batch_size=args.batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(mnist_testset, batch_size=args.test_batch_size, shuffle=True)

In [7]:
from torch import nn, optim
import torch.nn.functional as F
torch.set_default_tensor_type(torch.cuda.FloatTensor)

In [8]:
class Classifier(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [9]:
model = Classifier()
model = model.to(device)
optimizer = optim.SGD(model.parameters(), lr=args.lr)
for epoch in range(1, args.epochs + 1):
    model.train()
    for batch_idx, (data, target) in enumerate(federated_train_loader): # iterate through each worker's dataset
        
        model.send(data.location) #send the model to the right location
        
        data, target = data.to(device), target.to(device)
        
        optimizer.zero_grad() # 1) erase previous gradients (if they exist)
        output = model(data)  # 2) make a prediction
        loss = F.nll_loss(output, target)  # 3) calculate how much we missed
        loss.backward()  # 4) figure out which weights caused us to miss
        optimizer.step()  # 5) change those weights
        model.get()  # get the model back (with gradients)
        
        if batch_idx % args.log_interval == 0:
            loss = loss.get() #get the loss back
            print('Epoch: {} [Training: {:.0f}%]\tLoss: {:.6f}'.format(epoch, 100. * batch_idx / len(federated_train_loader), loss.item()))
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            
            data, target = data.to(device), target.to(device)
            output = model(data)
            
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(test_loss, correct, len(test_loader.dataset), 100. * correct / len(test_loader.dataset)))
    
if (args.save_model):
    torch.save(model.state_dict(), "mnist_cnn.pt")

Epoch: 1 [Training: 0%]	Loss: 2.307303
Epoch: 1 [Training: 3%]	Loss: 2.152380
Epoch: 1 [Training: 6%]	Loss: 1.851272
Epoch: 1 [Training: 10%]	Loss: 1.258177
Epoch: 1 [Training: 13%]	Loss: 0.785922
Epoch: 1 [Training: 16%]	Loss: 0.752820
Epoch: 1 [Training: 19%]	Loss: 0.389547
Epoch: 1 [Training: 22%]	Loss: 0.668079
Epoch: 1 [Training: 26%]	Loss: 0.556559
Epoch: 1 [Training: 29%]	Loss: 0.298134
Epoch: 1 [Training: 32%]	Loss: 0.709528
Epoch: 1 [Training: 35%]	Loss: 0.243506
Epoch: 1 [Training: 38%]	Loss: 0.295973
Epoch: 1 [Training: 42%]	Loss: 0.437727
Epoch: 1 [Training: 45%]	Loss: 0.444640
Epoch: 1 [Training: 48%]	Loss: 0.462919
Epoch: 1 [Training: 51%]	Loss: 0.377572
Epoch: 1 [Training: 54%]	Loss: 0.231817
Epoch: 1 [Training: 58%]	Loss: 0.164112
Epoch: 1 [Training: 61%]	Loss: 0.186264
Epoch: 1 [Training: 64%]	Loss: 0.267224
Epoch: 1 [Training: 67%]	Loss: 0.161641
Epoch: 1 [Training: 70%]	Loss: 0.223377
Epoch: 1 [Training: 74%]	Loss: 0.181561
Epoch: 1 [Training: 77%]	Loss: 0.177792
Epo

Epoch: 7 [Training: 16%]	Loss: 0.039918
Epoch: 7 [Training: 19%]	Loss: 0.014348
Epoch: 7 [Training: 22%]	Loss: 0.074999
Epoch: 7 [Training: 26%]	Loss: 0.004217
Epoch: 7 [Training: 29%]	Loss: 0.026987
Epoch: 7 [Training: 32%]	Loss: 0.027636
Epoch: 7 [Training: 35%]	Loss: 0.015602
Epoch: 7 [Training: 38%]	Loss: 0.088973
Epoch: 7 [Training: 42%]	Loss: 0.025428
Epoch: 7 [Training: 45%]	Loss: 0.015774
Epoch: 7 [Training: 48%]	Loss: 0.088870
Epoch: 7 [Training: 51%]	Loss: 0.054397
Epoch: 7 [Training: 54%]	Loss: 0.055844
Epoch: 7 [Training: 58%]	Loss: 0.004940
Epoch: 7 [Training: 61%]	Loss: 0.016300
Epoch: 7 [Training: 64%]	Loss: 0.004413
Epoch: 7 [Training: 67%]	Loss: 0.080199
Epoch: 7 [Training: 70%]	Loss: 0.011111
Epoch: 7 [Training: 74%]	Loss: 0.062064
Epoch: 7 [Training: 77%]	Loss: 0.109649
Epoch: 7 [Training: 80%]	Loss: 0.097691
Epoch: 7 [Training: 83%]	Loss: 0.021317
Epoch: 7 [Training: 86%]	Loss: 0.023506
Epoch: 7 [Training: 90%]	Loss: 0.015856
Epoch: 7 [Training: 93%]	Loss: 0.066526


In [10]:
print("Accuracy Obtained {:.4f}%".format( 100. * correct / len(test_loader.dataset)))

Accuracy Obtained 98.8200%
